In [2]:
# Install ReportLab
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.3 MB/s eta 0:00:00


In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Enhanced ACI 318-19 RC Beam Design Script — REVISED MASTER (full)
==================================================================

Directly revised from your MASTER.

Adds:
• Plan View page (optional JPG) after a static TOC.
• Clean, signed Vu/Mu diagrams for the governing strength combo + service deflection overlay.
• Correct shear-envelope design figure (plots |Vu| vs positive φVc and φ(Vc+Vs) only).
• Cross-section drawing with visible stirrups cage + nicer palette; optional stirrup spacing note.
• Flexure table column widths and 8pt font so it fits one page; repeat header row.
• Report header block (no page numbers) and references page.
• Robust service deflection with Branson Ie on D+L (zeroed span-by-span).

Assumptions:
• Units: kips and inches. UDL input in kip/ft → converted to kip/in.
• Default stirrups: 2-leg #4; code min & max spacings enforced; supports get the tighter spacing.
• Normal-weight concrete unless lam ≠ 1.0.
"""

from __future__ import annotations

import os
import math
import datetime as dt
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Any, Callable

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon

# ---------------------------------------------------------------------------
# ReportLab imports
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import (
    BaseDocTemplate, Frame, PageTemplate, Paragraph, Table, TableStyle,
    Spacer, Image as RLImage, PageBreak, Flowable
)
from reportlab.pdfgen import canvas as rlcanvas
from reportlab.lib.utils import ImageReader
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# ---------------------------------------------------------------------------
# Constants and bar data

FT2IN = 12.0
LOGO_DEFAULT = "/content/stvlogo23.png"

# Try to register Courier New; fallback to Courier
try:
    pdfmetrics.registerFont(TTFont('CourierNew', r'C:/Windows/Fonts/cour.ttf'))
    MONO = 'CourierNew'
except Exception:
    MONO = 'Courier'

# Bar diameters and areas (inches, square inches)
BAR_DB = {
    '#3': 0.375, '#4': 0.500, '#5': 0.625, '#6': 0.750,
    '#7': 0.875, '#8': 1.000, '#9': 1.128, '#10': 1.270, '#11': 1.410
}
BAR_AREA = {
    '#3': 0.11, '#4': 0.20, '#5': 0.31, '#6': 0.44, '#7': 0.60,
    '#8': 0.79, '#9': 1.00, '#10': 1.27, '#11': 1.56
}

STD_STIRRUPS = np.array([4.0, 6.0, 8.0, 10.0, 12.0])  # in

# ---------------------------------------------------------------------------
# Data models
@dataclass
class Material:
    fpc: float
    fy: float
    Es: float = 29000.0 * 1000.0
    lam: float = 1.0


@dataclass
class SpanLoad:
    wD: float
    wL: float
    point_loads: List[Tuple[float, float, str]] = field(default_factory=list)
    # (P, x(ft), 'D' or 'L')


@dataclass
class Span:
    L_ft: float
    load: SpanLoad


@dataclass
class BeamInput:
    spans: List[Span]
    joints: List[str]  # 'fixed' or 'pinned'; len = n_spans + 1


@dataclass
class Section:
    b: float
    h: float
    cover: float

    def d(self, db: float) -> float:
        # Effective depth to centroid of one bottom layer (assumes #3 stirrup ~ 3/8")
        return self.h - self.cover - 0.375 - 0.5 * db


# ---------------------------------------------------------------------------
# User input helpers
def ask_string(prompt: str, default: str = "") -> str:
    s = input(f"{prompt} [{default}]: ").strip()
    return s or default


def ask_float(prompt: str, default: float) -> float:
    while True:
        s = input(f"{prompt} [{default}]: ").strip()
        if s == "":
            return float(default)
        try:
            return float(s)
        except ValueError:
            print(" → Please enter a valid number.")


def ask_int(prompt: str, default: int, min_value: int | None = None, max_value: int | None = None) -> int:
    while True:
        s = input(f"{prompt} [{default}]: ").strip()
        if s == "":
            v = int(default)
        else:
            try:
                v = int(s)
            except ValueError:
                print(" → Please enter a whole number.")
                continue
        if min_value is not None and v < min_value:
            print(f" → Please enter a number ≥ {min_value}.")
            continue
        if max_value is not None and v > max_value:
            print(f" → Please enter a number ≤ {max_value}.")
            continue
        return v


def ask_choice(prompt: str, choices: List[str], default: str) -> str:
    choices_str = "/".join(choices)
    cset = [c.lower() for c in choices]
    while True:
        result = input(f"{prompt} ({choices_str}) [{default}]: ").strip().lower()
        if not result:
            return default.lower()
        if result in cset:
            return result
        print(f" → Please choose one of: {', '.join(choices)}")


# ---------------------------------------------------------------------------
# Load combinations framework (simple; envelope for strength)

# ---------------------------------------------------------------------------
# Load combinations framework (supports default or user-defined)
from typing import NamedTuple

class ComboDef(NamedTuple):
    name: str
    factors: Dict[str, float]  # e.g., {'D':1.2, 'L':1.6}

def _make_combo_fns(combo: ComboDef) -> Tuple[str, Callable[[float, float], float], Callable[[float, float], float]]:
    """
    Builds (name, f_udl, f_P) from factor dictionary.
    Currently recognizes 'D' and 'L' symbols. Unrecognized symbols are ignored
    (safe forward-compatibility if you later add more load components).
    """
    fD = combo.factors.get('D', 0.0)
    fL = combo.factors.get('L', 0.0)
    name = combo.name
    f_udl = lambda wD, wL: fD * wD + fL * wL
    f_P   = lambda PD, PL: fD * PD + fL * PL
    return (name, f_udl, f_P)

def pretty_combo(combo: ComboDef) -> str:
    parts = []
    for k, v in combo.factors.items():
        if abs(v) < 1e-12:
            continue
        coef = f"{v:g}"
        parts.append(f"{coef}{k}")
    return " + ".join(parts) if parts else "0"

# Default strength set (unchanged logic)
DEFAULT_COMBOS: List[ComboDef] = [
    ComboDef("U1: 1.4D",            {'D': 1.4, 'L': 0.0}),
    ComboDef("U2: 1.2D + 1.6L",     {'D': 1.2, 'L': 1.6}),
]

# ---------------------------------------------------------------------------
# Moment distribution (Hardy Cross)
@dataclass
class MDMember:
    L_in: float
    EI: float
    far_is_fixed: bool
    near_is_fixed: bool
    FEM_near: float = 0.0
    FEM_far: float = 0.0
    M_near: float = 0.0
    M_far: float = 0.0

    @property
    def k_near(self) -> float:
        return (4.0 if self.far_is_fixed else 3.0) * self.EI / self.L_in

    @property
    def k_far(self) -> float:
        return (4.0 if self.near_is_fixed else 3.0) * self.EI / self.L_in

    @property
    def CO_to_far(self) -> float:
        return 0.5 if self.far_is_fixed else 0.0

    @property
    def CO_to_near(self) -> float:
        return 0.5 if self.near_is_fixed else 0.0


def compute_FEM_udl(w_kipin: float, L_in: float) -> Tuple[float, float]:
    M = w_kipin * L_in ** 2 / 12.0
    return (M, M)


def compute_FEM_point(P_kip: float, a_in: float, L_in: float) -> Tuple[float, float]:
    b = L_in - a_in
    M_near = P_kip * a_in * b ** 2 / L_in ** 2
    M_far  = P_kip * a_in ** 2 * b / L_in ** 2
    return (M_near, M_far)


def build_members_combo(beam: BeamInput, E: float, I: float,
                        f_udl: Callable[[float, float], float],
                        f_P: Callable[[float, float], float]) -> List[MDMember]:
    members: List[MDMember] = []
    for i, sp in enumerate(beam.spans):
        L = sp.L_ft * FT2IN
        w = f_udl(sp.load.wD, sp.load.wL) / FT2IN  # kip/in
        near_fixed = beam.joints[i].lower() == 'fixed'
        far_fixed  = beam.joints[i + 1].lower() == 'fixed'
        m = MDMember(L_in=L, EI=E * I, far_is_fixed=far_fixed, near_is_fixed=near_fixed)
        FEM_n = 0.0
        FEM_f = 0.0
        if abs(w) > 1e-12:
            fn, ff = compute_FEM_udl(w, L)
            FEM_n += fn
            FEM_f += ff
        for (P, xft, typ) in sp.load.point_loads:
            PD = P if typ.upper() == 'D' else 0.0
            PL = P if typ.upper() == 'L' else 0.0
            P_combo = f_P(PD, PL)
            fn, ff = compute_FEM_point(P_combo, xft * FT2IN, L)
            FEM_n += fn
            FEM_f += ff
        m.FEM_near = m.M_near = FEM_n
        m.FEM_far  = m.M_far  = FEM_f
        members.append(m)
    return members


def moment_distribution_members(members: List[MDMember],
                                joints_types: List[str],
                                tol: float = 1e-6,
                                maxit: int = 500) -> List[MDMember]:
    nJ = len(joints_types)
    joints: List[List[Tuple[int, bool]]] = [[] for _ in range(nJ)]
    for i, m in enumerate(members):
        joints[i].append((i, True))
        joints[i + 1].append((i, False))
    for _ in range(maxit):
        max_unbal = 0.0
        for j in range(1, nJ - 1):
            if joints_types[j].lower() == 'pinned':
                continue
            K_sum = 0.0
            M_unbal = 0.0
            for (idx, is_near) in joints[j]:
                m = members[idx]
                if is_near:
                    K_sum += m.k_near
                    M_unbal += m.M_near
                else:
                    K_sum += m.k_far
                    M_unbal += m.M_far
            if abs(K_sum) < 1e-12:
                continue
            max_unbal = max(max_unbal, abs(M_unbal))
            for (idx, is_near) in joints[j]:
                m = members[idx]
                if is_near:
                    DF = m.k_near / K_sum
                    delta = -M_unbal * DF
                    m.M_near += delta
                    m.M_far  += m.CO_to_far * delta
                else:
                    DF = m.k_far / K_sum
                    delta = -M_unbal * DF
                    m.M_far  += delta
                    m.M_near += m.CO_to_near * delta
        if max_unbal < tol:
            break
    return members


# ---------------------------------------------------------------------------
# Diagram computation
@dataclass
class Diagrams:
    x_in: np.ndarray
    V_kip: np.ndarray
    M_kipin: np.ndarray
    w_in: np.ndarray


def get_span_loads_combo(sp: Span,
                         f_udl: Callable[[float, float], float],
                         f_P: Callable[[float, float], float]) -> Tuple[float, List[Tuple[float, float]]]:
    w = f_udl(sp.load.wD, sp.load.wL) / FT2IN
    pts: List[Tuple[float, float]] = []
    for (P, xft, t) in sp.load.point_loads:
        PD = P if t.upper() == 'D' else 0.0
        PL = P if t.upper() == 'L' else 0.0
        pts.append((f_P(PD, PL), xft * FT2IN))
    return w, pts


def compute_diagrams_combo(beam: BeamInput,
                           members: List[MDMember],
                           E: float,
                           I: float,
                           f_udl: Callable[[float, float], float],
                           f_P: Callable[[float, float], float],
                           n: int = 1000) -> Diagrams:
    all_x: List[np.ndarray] = []
    all_V: List[np.ndarray] = []
    all_M: List[np.ndarray] = []
    all_w: List[np.ndarray] = []
    x_global = 0.0
    for sp, mem in zip(beam.spans, members):
        L = sp.L_ft * FT2IN
        w, pts = get_span_loads_combo(sp, f_udl, f_P)
        M_left, M_right = mem.M_near, mem.M_far
        sum_P_moment = sum(P * (L - a) for (P, a) in pts)
        R_left = (w * L * L / 2.0 + sum_P_moment - M_left + M_right) / L
        x_local = np.linspace(0, L, n)
        dx = x_local[1] - x_local[0]
        V = np.zeros(n)
        M = np.zeros(n)
        for i, x in enumerate(x_local):
            V[i] = R_left - w * x - sum(P for (P, a) in pts if x >= a)
            M[i] = R_left * x - w * x ** 2 / 2.0 - sum(P * (x - a) for (P, a) in pts if x >= a) - M_left
        # Elastic deflection (for completeness—unused for ultimate)
        M_EI = M / (E * I)
        theta = np.zeros(n)
        w_defl = np.zeros(n)
        for i in range(1, n):
            theta[i]   = theta[i - 1] + 0.5 * (M_EI[i] + M_EI[i - 1]) * dx
            w_defl[i]  = w_defl[i - 1] + 0.5 * (theta[i] + theta[i - 1]) * dx
        # Enforce w(L)=0 per span
        if abs(w_defl[-1]) > 1e-12:
            w_defl -= w_defl[-1] * x_local / L
        all_x.append(x_global + x_local)
        all_V.append(V)
        all_M.append(M)
        all_w.append(w_defl)
        x_global += L
    return Diagrams(
        x_in=np.concatenate(all_x),
        V_kip=np.concatenate(all_V),
        M_kipin=np.concatenate(all_M),
        w_in=np.concatenate(all_w)
    )


def envelope_ultimate(beam: BeamInput, E: float, I: float, combos: List[ComboDef], n: int = 1000) -> Tuple[Diagrams, List[str]]:
    """Envelope ultimate Vu/Mu over selected strength combinations."""
    x_ref = None
    V_en = None
    M_en = None
    used = []
    for combo in combos:
        name, f_udl, f_P = _make_combo_fns(combo)
        members = build_members_combo(beam, E, I, f_udl, f_P)
        members = moment_distribution_members(members, beam.joints, tol=1e-7, maxit=500)
        diag    = compute_diagrams_combo(beam, members, E, I, f_udl, f_P, n=n)
        if x_ref is None:
            x_ref = diag.x_in
            V_en  = np.abs(diag.V_kip)
            M_en  = np.abs(diag.M_kipin)
        else:
            V_en = np.maximum(V_en, np.abs(diag.V_kip))
            M_en = np.maximum(M_en, np.abs(diag.M_kipin))
        used.append(name)
    return Diagrams(x_in=x_ref, V_kip=V_en, M_kipin=M_en, w_in=np.zeros_like(x_ref)), used



def _diag_for_combo(beam: BeamInput, E: float, I: float, f_udl, f_P, n: int = 1000) -> Diagrams:
    members = build_members_combo(beam, E, I, f_udl, f_P)
    members = moment_distribution_members(members, beam.joints, tol=1e-7, maxit=500)
    return compute_diagrams_combo(beam, members, E, I, f_udl, f_P, n=n)



def governing_signed_combo(beam: BeamInput, E: float, I: float,
                           combos: List[ComboDef],
                           n: int = 1000,
                           criterion: str = "moment") -> tuple[Diagrams, str]:
    """
    Return the signed diagrams for the ultimate combo that governs
    (by max |M| if criterion='moment', else by max |V|).
    """
    best = None
    best_name = ""
    for combo in combos:
        name, f_udl, f_P = _make_combo_fns(combo)
        d = _diag_for_combo(beam, E, I, f_udl, f_P, n=n)
        score = float(np.max(np.abs(d.M_kipin))) if (criterion == "moment") else float(np.max(np.abs(d.V_kip)))
        if (best is None) or (score > best[0]):
            best = (score, d, name)
    assert best is not None
    return best[1], best[2]



# ---------------------------------------------------------------------------
# Flexural design
def beta1_aci318_19(fpc_psi: float) -> float:
    if fpc_psi <= 4000.0:
        return 0.85
    red = 0.05 * ((fpc_psi - 4000.0) / 1000.0)
    return max(0.65, 0.85 - red)


def phi_flexure(eps_t: float) -> float:
    if eps_t >= 0.005:
        return 0.90
    if eps_t <= 0.002:
        return 0.65
    return 0.65 + (eps_t - 0.002) * (0.90 - 0.65) / (0.005 - 0.002)


def As_min_flexure(b: float, d: float, fpc: float, fy: float) -> float:
    return max(3.0 * math.sqrt(fpc) / fy, 200.0 / fy) * b * d


def design_flexure(Mu_kipin: float, b: float, d: float, fpc: float, fy: float) -> Dict[str, Any]:
    if Mu_kipin <= 0:
        return {
            'As': 0.0, 'a': 0.0, 'c': 0.0, 'jd': d, 'eps_t': 0.01, 'phi': 0.90,
            'Mn': 0.0, 'phiMn': 0.0, 'iterations': 0, 'beta1': beta1_aci318_19(fpc),
        }
    Mu = Mu_kipin * 1000.0
    beta1 = beta1_aci318_19(fpc)
    jd = 0.9 * d
    As = Mu / (0.90 * fy * jd)
    for it in range(1, 100):
        a = As * fy / (0.85 * fpc * b)
        c = a / beta1
        jd = d - a / 2.0
        As_new = Mu / (0.90 * fy * jd)
        if abs(As_new - As) / max(As_new, 1e-9) < 1e-3:
            As = As_new
            break
        As = As_new
    a = As * fy / (0.85 * fpc * b)
    c = a / beta1
    eps_t = 0.003 * (d - c) / max(c, 1e-9)
    phi = phi_flexure(eps_t)
    Mn = As * fy * (d - a / 2.0) / 1000.0
    return {'As': As, 'a': a, 'c': c, 'jd': jd, 'eps_t': eps_t, 'phi': phi,
            'Mn': Mn, 'phiMn': phi * Mn, 'iterations': it, 'beta1': beta1}


# ---------------------------------------------------------------------------
# Multi-layer layout helpers (longitudinal bars)
def min_clear_spacing(db: float, agg: float = 0.75) -> float:
    # ACI 318 spacing minimum: max(db, 1.0", 4/3*agg)
    return max(db, 1.0, 4.0 * agg / 3.0)


def arrange_layers(b: float, cover: float, db: float, n: int,
                   stirrup_db: float = 0.375, agg: float = 0.75) -> Dict[str, Any]:
    """
    Greedy layout: fill first bottom layer to meet min clear spacing; overflow to second layer.
    Returns {'layers': [{'n': n1, 'y': y1, 's': s1}, {'n': n2, 'y': y2, 's': s2}], ...}
    y = centerline elevation of bars in that layer from bottom of section.
    """
    s_min = min_clear_spacing(db, agg)
    clear_w = b - 2 * (cover + stirrup_db)
    # capacity per layer
    if clear_w <= db:
        per_layer = 1
        s1 = clear_w  # degenerate
    else:
        # n bars produce total width n*db + (n-1)*s; set s >= s_min
        per_layer = max(1, int(math.floor((clear_w + s_min) / (db + s_min))))
        # Recompute actual spacing with that count
        if per_layer == 1:
            s1 = clear_w - db
        else:
            s1 = (clear_w - per_layer * db) / (per_layer - 1)
    n1 = min(n, per_layer)
    n2 = max(0, n - n1)
    # y-levels
    y1 = cover + stirrup_db + db / 2.0
    y2 = y1 + db + s_min  # simple vertical staggering
    layers = [{'n': n1, 'y': y1, 's': s1}]
    if n2 > 0:
        layers.append({'n': n2, 'y': y2, 's': s1})
    return {'layers': layers}


def effective_depth_from_layers(sec: Section, db: float, layout: Dict[str, Any]) -> float:
    num = 0.0
    den = 0.0
    for lay in layout['layers']:
        num += lay['n'] * lay['y']
        den += lay['n']
    y_cent = num / max(den, 1e-9) if den > 0 else sec.cover + 0.375 + db / 2.0
    return sec.h - y_cent


def choose_bars(As_req: float, prefer: List[str] | None = None, min_bars: int = 2) -> Dict[str, Any]:
    if prefer is None:
        prefer = ['#5', '#6', '#7', '#8', '#9', '#10', '#11']
    best: Tuple[int, str, float] | None = None
    for size in prefer:
        Ab = BAR_AREA[size]
        n = max(min_bars, int(math.ceil(As_req / Ab)))
        As_prov = n * Ab
        if best is None or n < best[0]:
            best = (n, size, As_prov)
    n, size, As_prov = best  # type: ignore
    return {'n': n, 'size': size, 'As_prov': As_prov, 'db': BAR_DB[size]}


def check_bar_spacing(b: float, cover: float, n: int, db: float) -> Dict[str, Any]:
    stirrup_db = 0.375
    clear_width = b - 2 * (cover + stirrup_db)
    if n <= 1:
        s = clear_width
    else:
        s = (clear_width - n * db) / (n - 1)
    s_min = min_clear_spacing(db, 0.75)
    return {'s_actual': s, 's_min': s_min, 'adequate': s >= s_min, 'clear_width': clear_width}


def development_length_tension(db: float, fy: float, fpc: float, cover: float, spacing: float) -> float:
    psi_t = 1.3  # conservative top-bar factor
    ld = (3.0 * fy * psi_t / (40.0 * math.sqrt(fpc))) * db
    return max(ld, 12.0)


# ---------------------------------------------------------------------------
# Shear design (zones)
def Vc_beam(bw: float, d: float, fpc: float, lam: float = 1.0) -> float:
    return 2.0 * lam * math.sqrt(fpc) * bw * d / 1000.0  # kips


def shear_schedule_three_zones(diag: Diagrams, sec: Section, mat: Material
                               ) -> Tuple[List[List[str]], Dict[str, Any]]:
    total_L_ft = diag.x_in[-1] / FT2IN
    z_len = total_L_ft / 3.0
    bounds = [0.0, z_len, 2 * z_len, 3 * z_len]
    xft = diag.x_in / FT2IN
    Vu_abs = np.abs(diag.V_kip)
    idx1 = (xft >= bounds[0]) & (xft <= bounds[1])
    idx2 = (xft >= bounds[1]) & (xft <= bounds[2])
    idx3 = (xft >= bounds[2]) & (xft <= bounds[3])
    Vu1 = float(np.max(Vu_abs[idx1])) if np.any(idx1) else 0.0
    Vu2 = float(np.max(Vu_abs[idx2])) if np.any(idx2) else 0.0
    Vu3 = float(np.max(Vu_abs[idx3])) if np.any(idx3) else 0.0
    d_use = sec.d(1.0)
    Vc = Vc_beam(sec.b, d_use, mat.fpc, mat.lam)
    phi_v = 0.75
    Av = 2.0 * BAR_AREA['#4']  # 2-leg #4
    Rmin = max(0.75 * math.sqrt(mat.fpc) * sec.b / mat.fy, 50.0 * sec.b / mat.fy)
    s_min_code = Av / Rmin if Rmin > 0 else 24.0
    s_max_code = min(d_use / 2.0, 24.0)

    def zone_spacing(Vu_zone: float) -> Tuple[float, float, bool]:
        if Vu_zone <= phi_v * 0.5 * Vc:
            s_req_strength = float('inf'); required = False
        elif Vu_zone <= phi_v * Vc:
            s_req_strength = float('inf'); required = True
        else:
            Vs_req = Vu_zone / phi_v - Vc
            if Vs_req <= 0:
                s_req_strength = float('inf'); required = True
            else:
                s_req_strength = Av * mat.fy * d_use / (Vs_req * 1000.0)
                required = True
        if math.isfinite(s_req_strength):
            s_req = min(s_req_strength, s_min_code, s_max_code)
        else:
            s_req = min(s_min_code, s_max_code)
        candidates = STD_STIRRUPS[STD_STIRRUPS <= s_req + 1e-9]
        s_prov = float(candidates[-1]) if len(candidates) > 0 else float(STD_STIRRUPS[0])
        return s_req, s_prov, required

    s1_req, s1_prov, req1 = zone_spacing(Vu1)
    s2_req, s2_prov, req2 = zone_spacing(Vu2)
    s3_req, s3_prov, req3 = zone_spacing(Vu3)

    tightest = min(s1_prov, s2_prov, s3_prov)
    loosest  = max(s1_prov, s2_prov, s3_prov)
    final_s1 = tightest
    final_s3 = tightest
    final_s2 = loosest

    rows: List[List[str]] = [[
        "Zone", "x_start (ft)", "x_end (ft)", "Vu,max (k)", "φVc (k)", "s_req (in)", "s_prov (in)", "Required?",
    ]]
    rows.append(["1 (Left)",  f"{bounds[0]:.2f}", f"{bounds[1]:.2f}", f"{Vu1:.2f}", f"{phi_v * Vc:.2f}", f"{s1_req:.1f}", f"{final_s1:.0f}", "Yes" if req1 else "Min"])
    rows.append(["2 (Mid)",   f"{bounds[1]:.2f}", f"{bounds[2]:.2f}", f"{Vu2:.2f}", f"{phi_v * Vc:.2f}", f"{s2_req:.1f}", f"{final_s2:.0f}", "Yes" if req2 else "Min"])
    rows.append(["3 (Right)", f"{bounds[2]:.2f}", f"{bounds[3]:.2f}", f"{Vu3:.2f}", f"{phi_v * Vc:.2f}", f"{s3_req:.1f}", f"{final_s3:.0f}", "Yes" if req3 else "Min"])

    meta = {'Vc': Vc, 'phi_v': phi_v, 'd_use': d_use, 'Av': Av,
            's_min_code': s_min_code, 's_max': s_max_code, 'z_len': z_len,
            's1_prov': final_s1, 's2_prov': final_s2, 's3_prov': final_s3,
            'bounds': bounds}
    return rows, meta


def spacing_piecewise_from_meta(x_ft: np.ndarray, meta: Dict[str, Any]) -> np.ndarray:
    b0, b1, b2, b3 = meta['bounds']
    s_arr = np.zeros_like(x_ft, dtype=float)
    s1 = meta['s1_prov']; s2 = meta['s2_prov']; s3 = meta['s3_prov']
    s_arr[(x_ft >= b0) & (x_ft <= b1)] = s1
    s_arr[(x_ft >  b1) & (x_ft <= b2)] = s2
    s_arr[(x_ft >  b2) & (x_ft <= b3)] = s3
    s_arr[s_arr == 0.0] = s3
    return s_arr


def shear_capacity_lines(diag: Diagrams, sec: Section, mat: Material, shear_meta: Dict[str, Any]
                         ) -> Tuple[np.ndarray, np.ndarray]:
    phi_v = shear_meta['phi_v']
    d_use = shear_meta['d_use']
    Av    = shear_meta['Av']
    Vc    = shear_meta['Vc']
    x_ft  = diag.x_in / FT2IN
    phiVc_line = np.full_like(x_ft, phi_v * Vc)
    s_x_in = spacing_piecewise_from_meta(x_ft, shear_meta)
    Vs_lb  = (Av * mat.fy * d_use) / np.maximum(s_x_in, 1e-9)
    Vs_k   = Vs_lb / 1000.0
    phi_total = phi_v * (Vc + Vs_k)
    return phiVc_line, phi_total


# ---------------------------------------------------------------------------
# Plotting
def draw_support(ax, x: float, y: float, kind: str, scale: float = 1.0) -> None:
    if kind.lower() == 'pinned':
        tri = Polygon([[x - 0.4 * scale, y], [x + 0.4 * scale, y], [x, y - 0.35 * scale]],
                      closed=True, facecolor='white', edgecolor='k', linewidth=1.5)
        ax.add_patch(tri)
        ax.plot([x - 0.5 * scale, x + 0.5 * scale], [y - 0.35 * scale, y - 0.35 * scale], 'k-', linewidth=1.0)
    else:
        rect = Rectangle((x - 0.15 * scale, y - 0.4 * scale), 0.3 * scale, 0.8 * scale,
                         facecolor='white', edgecolor='k', linewidth=1.5)
        ax.add_patch(rect)
        for i in range(5):
            y_i = y - 0.4 * scale + i * 0.2 * scale
            ax.plot([x - 0.15 * scale, x - 0.25 * scale], [y_i, y_i - 0.1 * scale], 'k-', linewidth=1.0)


def draw_schematic(beam: BeamInput, out_png: str) -> None:
    total_L = sum(sp.L_ft for sp in beam.spans)
    fig, ax = plt.subplots(figsize=(10.0, 3.0))
    y0 = 0.0
    xcur = 0.0
    ax.plot([0, total_L], [y0, y0], 'k-', linewidth=4)
    draw_support(ax, 0.0, y0, beam.joints[0], scale=0.8)
    for i, sp in enumerate(beam.spans):
        xcur += sp.L_ft
        draw_support(ax, xcur, y0, beam.joints[i + 1], scale=0.8)
    xcur = 0.0
    for sp in beam.spans:
        w_total = sp.load.wD + sp.load.wL
        if w_total > 0:
            rect = Rectangle((xcur, y0 + 0.1), sp.L_ft, 0.25,
                             facecolor='#cfe8ff', edgecolor='#2b7bff', linewidth=0.8, alpha=0.6)
            ax.add_patch(rect)
            n_arrows = max(8, int(sp.L_ft // 3))
            xs = np.linspace(xcur + 0.5, xcur + sp.L_ft - 0.5, n_arrows)
            for xx in xs:
                ax.annotate('', xy=(xx, y0 + 0.12), xytext=(xx, y0 + 0.60),
                            arrowprops=dict(arrowstyle='->', lw=1.2, color='#2b7bff'))
            ax.text(xcur + sp.L_ft / 2, y0 + 0.75, f"w = {w_total:.2f} k/ft",
                    ha='center', va='bottom', fontsize=9, color='#2b7bff')
        for (P, x, t) in sp.load.point_loads:
            xx = xcur + x
            ax.annotate('', xy=(xx, y0 + 0.05), xytext=(xx, y0 + 1.2),
                        arrowprops=dict(arrowstyle='->', lw=2.0, color='#e53935'))
            ax.text(xx, y0 + 1.3, f"P = {P:.1f} k ({t})",
                    ha='center', va='bottom', fontsize=9, color='#e53935')
        y_dim = y0 - 1.0
        ax.annotate('', xy=(xcur, y_dim), xytext=(xcur + sp.L_ft, y_dim),
                    arrowprops=dict(arrowstyle='<->', lw=1.0, color='black'))
        ax.text(xcur + sp.L_ft / 2, y_dim - 0.15, f"L = {sp.L_ft:.1f} ft",
                ha='center', va='top', fontsize=9, weight='bold')
        xcur += sp.L_ft
    ax.set_xlim(-1.5, total_L + 1.5)
    ax.set_ylim(-1.5, 2.0)
    ax.axis('off')
    ax.set_aspect('equal')
    ax.set_title('Beam Schematic: Supports and Loads', fontsize=12, weight='bold')
    plt.tight_layout()
    plt.savefig(out_png, dpi=250, bbox_inches='tight')
    plt.close()


def plot_diagrams(diag, base: str, combo_label: str = "Factored") -> tuple[str, str, str]:
    """Signed Vu & Mu plots + service deflection; titles boldened."""
    shear_png  = base + '_shear.png'
    moment_png = base + '_moment.png'
    defl_png   = base + '_deflect.png'
    xft = diag.x_in / FT2IN

    # Shear
    fig, ax = plt.subplots(figsize=(10, 3.5))
    ax.plot(xft, diag.V_kip, linewidth=2.2, label=f'Vu (kips) — {combo_label}', color='#1565c0')
    ax.axhline(0, color='k', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.fill_between(xft, 0.0, diag.V_kip, alpha=0.20, color='#90caf9')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Distance (ft)')
    ax.set_ylabel('Shear V (kips)')
    ax.set_title('Shear Force Diagram', fontsize=12, fontweight='bold')
    ax.legend(loc='best', frameon=True)
    plt.tight_layout(); plt.savefig(shear_png, dpi=220, bbox_inches='tight'); plt.close()

    # Moment
    M_kipft = diag.M_kipin / 12.0
    fig, ax = plt.subplots(figsize=(10, 3.5))
    ax.plot(xft, M_kipft, linewidth=2.2, label=f'Mu (kip-ft) — {combo_label}', color='#2e7d32')
    ax.axhline(0, color='k', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.fill_between(xft, 0.0, M_kipft, alpha=0.20, color='#a5d6a7')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Distance (ft)')
    ax.set_ylabel('Moment M (kip-ft)')
    ax.set_title('Bending Moment Diagram (Positive = Tension Bottom)', fontsize=12, fontweight='bold')
    ax.legend(loc='best', frameon=True)
    plt.tight_layout(); plt.savefig(moment_png, dpi=220, bbox_inches='tight'); plt.close()

    # Deflection
    fig, ax = plt.subplots(figsize=(10, 3.5))
    ax.plot(xft, diag.w_in, linewidth=2.2, label='Deflection (in)', color='#6a1b9a')
    ax.axhline(0, color='k', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.fill_between(xft, 0.0, diag.w_in, alpha=0.18, color='#ce93d8')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('Distance (ft)')
    ax.set_ylabel('Deflection (in)')
    ax.set_title('Deflection Diagram (service)', fontsize=12, fontweight='bold')

    if len(diag.w_in) > 0:
        idx = int(np.argmax(np.abs(diag.w_in)))
        ax.plot(xft[idx], diag.w_in[idx], 'o', markersize=6, color='#4a148c')
        ax.annotate(
            f"Max: {abs(diag.w_in[idx]):.3f} in\n@ x={xft[idx]:.1f} ft",
            xy=(xft[idx], diag.w_in[idx]),
            xytext=(xft[idx], diag.w_in[idx] * 0.5),
            fontsize=9, ha='center',
            bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7),
            arrowprops=dict(arrowstyle='->', lw=1.0, color='#4a148c'),
        )

    ax.legend(loc='best', frameon=True)
    plt.tight_layout(); plt.savefig(defl_png, dpi=220, bbox_inches='tight'); plt.close()
    return shear_png, moment_png, defl_png


# --- NEW: envelope shear design plot WITHOUT negative capacity curves -------------
def plot_shear_envelope_design(diag_env: Diagrams, sec: Section, mat: Material,
                               shear_meta: Dict[str, Any], base: str) -> str:
    """
    Show |Vu| vs positive φVc and φ(Vc+Vs) only.
    """
    xft = diag_env.x_in / FT2IN
    phi_v = shear_meta['phi_v']
    d_use = shear_meta['d_use']
    Av    = shear_meta['Av']
    Vc    = shear_meta['Vc']
    phiVc_line = np.full_like(xft, phi_v * Vc)

    # Piecewise provided spacing → Vs(x)
    s_x_in = spacing_piecewise_from_meta(xft, shear_meta)
    Vs_lb  = (Av * mat.fy * d_use) / np.maximum(s_x_in, 1e-9)   # lb
    Vs_k   = Vs_lb / 1000.0
    phiTotal = phi_v * (Vc + Vs_k)

    fig, ax = plt.subplots(figsize=(10, 3.8))
    ax.plot(xft, np.abs(diag_env.V_kip), label='Vu (kips)', linewidth=2.2, color='#1e88e5')
    ax.plot(xft, phiVc_line, label='φVc (kips)', linewidth=2.0, linestyle='--', color='#ef6c00')
    ax.plot(xft, phiTotal, label='φ(Vc + Vs) (kips)', linewidth=2.2, color='#2e7d32')

    # annotate at worst |Vu|
    i_crit = int(np.argmax(np.abs(diag_env.V_kip)))
    xcrit = xft[i_crit]
    Vucrit = float(np.abs(diag_env.V_kip[i_crit]))
    s_here = float(spacing_piecewise_from_meta(np.array([xcrit]), shear_meta)[0])
    Vs_k_here  = float((Av * mat.fy * d_use) / (s_here * 1000.0))

    ax.plot([xcrit], [Vucrit], 'o', color='#c62828', ms=6)
    ax.annotate(
        f"x = {xcrit:.2f} ft\nVu = {Vucrit:.2f} k\ns = {s_here:.0f}\" → Vs ≈ {Vs_k_here:.2f} k",
        xy=(xcrit, Vucrit), xytext=(xcrit, Vucrit * 0.70),
        bbox=dict(boxstyle='round', facecolor='#fff59d', alpha=0.9),
        arrowprops=dict(arrowstyle='->', lw=1.0, color='#c62828'),
        fontsize=9,
    )

    ax.set_xlabel('Distance (ft)')
    ax.set_ylabel('Shear (kips)')
    ax.set_title('Shear Design: |Vu| vs φVc and φ(Vc+Vs)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.30)
    ax.legend(loc='best', frameon=True)

    out_png = base + '_shear_design.png'
    plt.tight_layout(); plt.savefig(out_png, dpi=230, bbox_inches='tight'); plt.close()
    return out_png


# --- IMPROVED CROSS-SECTION (shows stirrup cage) ---------------------------------
def draw_cross_section(sec: Section,
                       bot_bars: Dict[str, Any],
                       top_bars: Dict[str, Any],
                       out_png: str,
                       bot_layout: Dict[str, Any] | None = None,
                       top_layout: Dict[str, Any] | None = None,
                       stirrup_text: str | None = None) -> None:
    """
    Nicer palette and visible stirrups cage with hooks indication.
    """
    fig, ax = plt.subplots(figsize=(5.0, 5.0))
    # concrete
    ax.add_patch(Rectangle((0, 0), sec.b, sec.h, facecolor='#e6e6e6', edgecolor='black', linewidth=1.6))
    # cover shading
    cov = sec.cover
    cov_col = '#f4e4c1'
    for (x, y, w, h) in [(0, sec.h-cov, sec.b, cov), (0, 0, sec.b, cov),
                         (0, 0, cov, sec.h), (sec.b - cov, 0, cov, sec.h)]:
        ax.add_patch(Rectangle((x, y), w, h, facecolor=cov_col, edgecolor='none', alpha=0.6))

    # stirrup rectangle
    stirrup_db = 0.375
    sx0, sy0 = cov + stirrup_db/2, cov + stirrup_db/2
    sx1, sy1 = sec.b - cov - stirrup_db/2, sec.h - cov - stirrup_db/2
    ax.add_patch(Rectangle((sx0, sy0), sx1 - sx0, sy1 - sy0,
                           fill=False, edgecolor='#2c7fb8', linewidth=2.0, linestyle='-'))
    # tiny hook marks
    ax.plot([sx0, sx0 + 0.7], [sy0, sy0 + 0.7], color='#2c7fb8', lw=1.5)
    ax.plot([sx0, sx0 + 0.7], [sy1, sy1 - 0.7], color='#2c7fb8', lw=1.5)

    # helper to place a layer
    def place_layer(n: int, y_center: float, db: float, label: str):
        if n <= 0: return
        if n == 1:
            xs = [sec.b / 2.0]
        else:
            x_start = cov + stirrup_db + db / 2.0
            x_end   = sec.b - cov - stirrup_db - db / 2.0
            xs = np.linspace(x_start, x_end, n)
        for x in xs:
            ax.add_patch(Circle((x, y_center), db/2.0, facecolor='#8b0000', edgecolor='black', linewidth=0.8))
        ax.text(sec.b + 0.8, y_center, label, va='center', fontsize=9, weight='bold')

    # bottom layers
    if bot_bars['n'] > 0:
        if bot_layout is None:
            yb = cov + stirrup_db + bot_bars['db']/2.0
            place_layer(bot_bars['n'], yb, bot_bars['db'], f"{bot_bars['n']}-{bot_bars['size']} bot")
        else:
            remaining = bot_bars['n']
            for i, lay in enumerate(bot_layout['layers']):
                n_i = min(remaining, lay['n'])
                place_layer(n_i, lay['y'], bot_bars['db'], f"{n_i}-{bot_bars['size']} bot L{i+1}")
                remaining -= n_i

    # top layers
    if top_bars['n'] > 0:
        if top_layout is None:
            yt = sec.h - cov - stirrup_db - top_bars['db']/2.0
            place_layer(top_bars['n'], yt, top_bars['db'], f"{top_bars['n']}-{top_bars['size']} top")
        else:
            remaining = top_bars['n']
            for i, lay in enumerate(top_layout['layers']):
                y_i = sec.h - lay['y']
                n_i = min(remaining, lay['n'])
                place_layer(n_i, y_i, top_bars['db'], f"{n_i}-{top_bars['size']} top L{i+1}")
                remaining -= n_i

    # dimensions
    ax.annotate('', xy=(0, -1.5), xytext=(sec.b, -1.5), arrowprops=dict(arrowstyle='<->', lw=1.2))
    ax.text(sec.b/2, -2.0, f'b = {sec.b:.1f}"', ha='center', fontsize=10, weight='bold')
    ax.annotate('', xy=(-1.5, 0), xytext=(-1.5, sec.h), arrowprops=dict(arrowstyle='<->', lw=1.2))
    ax.text(-2.3, sec.h/2, f'h = {sec.h:.1f}"', rotation=90, va='center', fontsize=10, weight='bold')

    # effective depth marker (bottom)
    d_eff = sec.d(bot_bars['db'])
    ax.plot([sec.b + 1, sec.b + 1], [0, d_eff], 'g-', linewidth=2)
    ax.plot([sec.b + 0.7, sec.b + 1.3], [d_eff, d_eff], 'g-', linewidth=2)
    ax.text(sec.b + 1.5, d_eff/2, f"d = {d_eff:.2f}\"", rotation=90, va='center', fontsize=9, color='green', weight='bold')

    # stirrup note
    if stirrup_text:
        ax.text(sec.b/2, sec.h + 1.1, stirrup_text, ha='center', va='bottom',
                fontsize=10, color='#2c7fb8', weight='bold')

    ax.set_xlim(-4, sec.b + 6)
    ax.set_ylim(-3, sec.h + 3)
    ax.set_aspect('equal'); ax.axis('off')
    ax.set_title('Reinforced Concrete Section', fontsize=12, fontweight='bold')
    plt.tight_layout(); plt.savefig(out_png, dpi=250, bbox_inches='tight'); plt.close()


# ---------------------------------------------------------------------------
# Service deflection with cracked Ie (Branson)
def compute_service_deflection_with_cracking(
    beam: BeamInput,
    Ec: float,
    Ig: float,
    sec: Section,
    mat: Material,
    bot_bars: Dict[str, Any],
    top_bars: Dict[str, Any],
    n_points_per_span: int = 1000,
) -> Tuple[np.ndarray, np.ndarray]:
    """Returns x (in) and w(x) (in) zeroed at supports, under D+L using Branson Ie."""
    # Build service members
    members = build_members_combo(beam, Ec, Ig, lambda wD, wL: (wD + wL), lambda PD, PL: (PD + PL))
    members = moment_distribution_members(members, beam.joints, tol=1e-7, maxit=500)
    diag = compute_diagrams_combo(beam, members, Ec, Ig, lambda wD, wL: (wD + wL), lambda PD, PL: (PD + PL), n=n_points_per_span)
    M = diag.M_kipin * 1000.0  # lb-in
    # Section parameters
    yt  = sec.h / 2.0
    fr  = 7.5 * mat.lam * math.sqrt(mat.fpc)        # psi
    Mcr = fr * Ig / yt                               # lb-in
    # Cracked inertia (rectangular, single layer approx using bottom steel)
    rho = (max(bot_bars.get('As_prov', 0.0), 1e-6) / (sec.b * sec.h))
    n   = mat.Es / Ec
    kd  = math.sqrt(2 * rho * n + (rho * n) ** 2) - rho * n
    kd  = max(min(kd, sec.h - 0.5), 0.5)            # clamp
    Icr = sec.b * (sec.h * kd) ** 3 / 3.0 + max(bot_bars.get('As_prov', 1e-6), 1e-6) * (sec.h - kd) ** 2
    # Effective inertia
    Ma  = np.maximum(np.abs(M), 1e-6)
    Ie  = ((Mcr / Ma) ** 3) * Ig + (1 - (Mcr / Ma) ** 3) * Icr
    Ie  = np.minimum(Ig, np.maximum(Icr, Ie))
    # Integrate twice
    x = diag.x_in
    dx = x[1] - x[0]
    theta = np.zeros_like(x)
    w     = np.zeros_like(x)
    for i in range(1, len(x)):
        m_ei_i   = M[i]     / (Ec * Ie[i])
        m_ei_prv = M[i - 1] / (Ec * Ie[i - 1])
        theta[i] = theta[i - 1] + 0.5 * (m_ei_i + m_ei_prv) * dx
        w[i]     = w[i - 1] + 0.5 * (theta[i] + theta[i - 1]) * dx
    # Zero at each support more robustly
    offset_in = 0.0
    for sp in beam.spans:
        L_in = sp.L_ft * FT2IN
        start_idx = int(np.argmin(np.abs(x - offset_in)))
        end_idx   = int(np.argmin(np.abs(x - (offset_in + L_in))))
        if end_idx > start_idx:
            # subtract linear ramp so w(start)=0 and w(end)=0
            w_start = w[start_idx]; w_end = w[end_idx]
            seg = slice(start_idx, end_idx + 1)
            xi = x[seg]
            t  = (xi - xi[0]) / max(xi[-1] - xi[0], 1e-9)
            w[seg] = w[seg] - (w_start + (w_end - w_start) * t)
        offset_in += L_in
    # Shift so first support is zero
    w -= w[int(np.argmin(np.abs(x - 0.0)))]
    return x, w


def cracking_moment_rect(b: float, h: float, fpc: float, lam: float = 1.0) -> float:
    fr = 7.5 * lam * math.sqrt(fpc)
    Ig = b * h**3 / 12.0
    yt = h / 2.0
    Mcr_lb_in = fr * Ig / yt
    return Mcr_lb_in / 1000.0  # kip-in


# ---------------------------------------------------------------------------
# PDF report generation
class TitleBlock(Flowable):
    """Header title block (no footer page numbers)."""
    def __init__(self, project: str, subject: str, by: str, checked: str, sheet: str, date: str, projno: str, logo_path: str):
        super().__init__()
        self.project = project; self.subject = subject
        self.by = by; self.checked = checked
        self.sheet = sheet; self.date = date; self.projno = projno
        self.logo_path = logo_path
        self.h = 0.9 * inch; self.w = 7.0 * inch

    def draw(self):
        c = self.canv; x, y = 0, 0
        c.saveState(); c.setLineWidth(1.0); c.rect(x, y, self.w, self.h)
        logo_w = 1.2 * inch
        c.line(x + logo_w, y, x + logo_w, y + self.h)
        if self.logo_path and os.path.exists(self.logo_path):
            try:
                img = ImageReader(self.logo_path)
                iw, ih = img.getSize()
                scale = min((logo_w - 10)/iw, (self.h - 10)/ih)
                c.drawImage(img, x + 5, y + (self.h - ih*scale)/2.0, width=iw*scale, height=ih*scale, mask='auto')
            except Exception:
                pass
        info_w = self.w - logo_w
        c.line(x + logo_w + info_w*0.7, y, x + logo_w + info_w*0.7, y + self.h)
        c.line(x + logo_w, y + self.h*0.6, x + self.w, y + self.h*0.6)
        c.line(x + logo_w, y + self.h*0.3, x + self.w, y + self.h*0.3)

        c.setFont(MONO, 8)
        c.drawString(x + logo_w + 5, y + self.h - 12, 'PROJECT:')
        c.drawString(x + logo_w + 60, y + self.h - 12, self.project[:35])
        c.drawString(x + logo_w + info_w*0.7 + 5, y + self.h - 12, 'DATE:')
        c.drawString(x + logo_w + info_w*0.7 + 40, y + self.h - 12, self.date)
        c.drawString(x + logo_w + 5, y + self.h * 0.45, 'SUBJECT:')
        c.drawString(x + logo_w + 60, y + self.h * 0.45, self.subject[:40])
        c.drawString(x + logo_w + info_w*0.7 + 5, y + self.h * 0.45, 'PROJ NO:')
        c.drawString(x + logo_w + info_w*0.7 + 40, y + self.h * 0.45, self.projno)
        c.drawString(x + logo_w + 5, y + 8, 'BY:')
        c.drawString(x + logo_w + 25, y + 8, self.by)
        c.drawString(x + logo_w + info_w * 0.35, y + 8, 'CHECKED:')
        c.drawString(x + logo_w + info_w * 0.35 + 55, y + 8, self.checked)
        c.restoreState()

    def wrap(self, availWidth: float, availHeight: float) -> Tuple[float, float]:
        return self.w, self.h


def create_page_header(project: str, subject: str, meta: Dict[str, Any]):
    """onPage header with downward shift; no footer/page number."""
    def header_func(canvas: rlcanvas.Canvas, doc: BaseDocTemplate) -> None:
        canvas.saveState()
        tb = TitleBlock(project, subject,
                        meta.get('By', ''), meta.get('Checked', ''),
                        f"{canvas.getPageNumber()}",
                        meta.get('Date', ''), meta.get('ProjectNo', ''),
                        meta.get('LogoPath', ''))
        canvas.translate(0.75 * inch, 9.3 * inch)
        tb.canv = canvas; tb.draw()
        canvas.restoreState()
    return header_func


# --- Helper to drop a plan-view image with centered title -----------------
def _plan_image_flowable(plan_path: str, max_w: float) -> RLImage | None:
    if not plan_path or not os.path.exists(plan_path):
        return None
    try:
        img = ImageReader(plan_path)
        iw, ih = img.getSize()
        scale = (max_w) / iw
        return RLImage(plan_path, width=iw*scale, height=ih*scale)
    except Exception:
        return None


def build_pdf_report(
    out_pdf: str,
    info: Dict[str, Any],
    diag, sec, mat,
    flex_table: List[List[str]],
    shear_table: List[List[str]],
    design_summary: List[str],
    schematic_png: str,
    shear_png: str,
    moment_png: str,
    defl_png: str,
    xsec_png: str,
    calc_table: List[List[str]],
    methodology: List[str],
    stirrup_summary: str,
    shear_design_png: str,
    combo_label: str = "Factored",
    plan_view_jpg: str | None = None,
    combos_table: List[List[str]] | None = None,   # ← NEW
) -> None:

    styles = getSampleStyleSheet()
    styles['Normal'].fontName = MONO; styles['Normal'].fontSize = 9; styles['Normal'].leading = 11
    if 'H1' not in styles:
        styles.add(ParagraphStyle(name='H1', parent=styles['Normal'], fontSize=13, leading=15,
                                  spaceAfter=10, textColor=colors.HexColor('#000080'),
                                  fontName='Helvetica-Bold'))
    if 'H2' not in styles:
        styles.add(ParagraphStyle(name='H2', parent=styles['Normal'], fontSize=11, leading=13,
                                  spaceAfter=8, fontName='Helvetica-Bold'))
    if 'Code' not in styles:
        styles.add(ParagraphStyle(name='Code', parent=styles['Normal'], fontSize=8, leading=10,
                                  leftIndent=12, fontName=MONO))

    doc = BaseDocTemplate(
        out_pdf, pagesize=letter,
        leftMargin=0.75 * inch, rightMargin=0.75 * inch,
        topMargin=2.4 * inch, bottomMargin=0.9 * inch
    )
    frame = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height, id='normal')
    doc.addPageTemplates([
        PageTemplate(id='main', frames=[frame],
                     onPage=create_page_header(info['Project'], info['Subject'], info))
    ])

    story: List[Any] = []

    # ---- TOC (static) ----
    toc_items = [
        "1. PROJECT SCOPE",
        "2. CODES AND STANDARDS",
        "3. ANALYSIS METHODOLOGY",
        "4. BEAM SCHEMATIC",
        "5. STRUCTURAL ANALYSIS RESULTS",
        "6. FLEXURAL DESIGN (ACI 318-19)",
        "7. SHEAR DESIGN AND STIRRUP SCHEDULE",
        "8. DETAILED CALCULATIONS",
        "9. REFERENCES & EQUATIONS",
        "10. CONCLUSIONS AND RECOMMENDATIONS",
    ]
    story.append(Paragraph('<b>TABLE OF CONTENTS</b>', styles['H1']))
    for item in toc_items:
        story.append(Paragraph(item, styles['Normal']))
    story.append(Spacer(1, 0.2 * inch))
    story.append(PageBreak())

    # ---- NEW PAGE 2: Plan View (if provided) ----
    if plan_view_jpg and os.path.exists(plan_view_jpg):
        story.append(Paragraph('BEAM PLAN VIEW', styles['H1']))
        img_flow = _plan_image_flowable(plan_view_jpg, doc.width)
        if img_flow:
            story.append(Spacer(1, 0.10 * inch))
            story.append(img_flow)
        story.append(PageBreak())

    # ---- 1. Scope ----
    story.append(Paragraph('1. PROJECT SCOPE', styles['H1']))
    story.append(Paragraph(info.get('Scope', ''), styles['Normal']))
    story.append(Spacer(1, 0.2 * inch))

    # ---- 2. Codes ----
    story.append(Paragraph('2. CODES AND STANDARDS', styles['H1']))
    story.append(Paragraph('• ACI 318-19 — Building Code Requirements for Structural Concrete', styles['Normal']))
    story.append(Paragraph('• ASCE 7 — Minimum Design Loads and Associated Criteria', styles['Normal']))
    story.append(Spacer(1, 0.2 * inch))

    # ---- 3. Methodology ----
    story.append(Paragraph('3. ANALYSIS METHODOLOGY', styles['H1']))
    for line in methodology:
        story.append(Paragraph(line, styles['Code']))
    story.append(Spacer(1, 0.2 * inch))
    # ---- 3.5 Load Combinations ----
    if combos_table:
        story.append(Paragraph('3.5 LOAD COMBINATIONS', styles['H1']))
        tcombo = Table(combos_table, hAlign='LEFT', repeatRows=1, colWidths=[2.3*inch, 3.9*inch])
        tcombo.setStyle(TableStyle([
            ('GRID', (0,0), (-1,-1), 0.5, colors.black),
            ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#4472C4')),
            ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
            ('FONTNAME', (0,0), (-1,-1), MONO),
            ('FONTSIZE', (0,0), (-1,-1), 8),
            ('ALIGN', (0,0), (-1,-1), 'LEFT'),
            ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor('#E7E6E6')]),
        ]))
        story.append(tcombo)
        story.append(Spacer(1, 0.2 * inch))

    # ---- 4. Schematic ----
    story.append(Paragraph('4. BEAM SCHEMATIC', styles['H1']))
    if os.path.exists(schematic_png):
        story.append(RLImage(schematic_png, width=6.5 * inch, height=2.0 * inch))
    story.append(Spacer(1, 0.15 * inch))

    # ---- 5. Analysis results ----
    story.append(Paragraph('5. STRUCTURAL ANALYSIS RESULTS', styles['H1']))
    story.append(Paragraph(f'Load combination shown: <b>{combo_label}</b>', styles['Normal']))
    if os.path.exists(shear_png):
        story.append(RLImage(shear_png, width=6.5 * inch, height=2.2 * inch))
    if os.path.exists(moment_png):
        story.append(RLImage(moment_png, width=6.5 * inch, height=2.2 * inch))
    if os.path.exists(defl_png):
        story.append(RLImage(defl_png, width=6.5 * inch, height=2.2 * inch))
    story.append(PageBreak())

    # ---- 6. Flexural design — MAKE IT FIT ----
    story.append(Paragraph('6. FLEXURAL DESIGN (ACI 318-19)', styles['H1']))
    flex_colw = [1.35*inch, 1.0*inch, 0.8*inch, 0.9*inch, 0.9*inch, 0.9*inch, 0.6*inch, 0.5*inch, 0.9*inch]
    t1 = Table(flex_table, hAlign='LEFT', colWidths=flex_colw, repeatRows=1)
    t1.setStyle(TableStyle([
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#4472C4')),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('FONTNAME', (0,0), (-1,-1), MONO),
        ('FONTSIZE', (0,0), (-1,-1), 8),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor('#E7E6E6')]),
    ]))
    story.append(t1)
    for line in design_summary:
        story.append(Paragraph(f"• {line}", styles['Normal']))
    story.append(PageBreak())

    # ---- 7. Shear design ----
    story.append(Paragraph('7. SHEAR DESIGN AND STIRRUP SCHEDULE', styles['H1']))
    t2 = Table(shear_table, hAlign='LEFT', repeatRows=1)
    t2.setStyle(TableStyle([
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor('#4472C4')),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('FONTNAME', (0,0), (-1,-1), MONO),
        ('FONTSIZE', (0,0), (-1,-1), 8),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('ROWBACKGROUNDS', (0,1), (-1,-1), [colors.white, colors.HexColor('#E7E6E6')]),
    ]))
    story.append(t2)
    if os.path.exists(shear_design_png):
        story.append(Spacer(1, 0.15 * inch))
        story.append(RLImage(shear_design_png, width=6.5 * inch, height=2.4 * inch))
    if os.path.exists(xsec_png):
        story.append(Spacer(1, 0.15 * inch))
        story.append(RLImage(xsec_png, width=4.2 * inch, height=3.4 * inch))
    story.append(PageBreak())

    # ---- 8. Calcs ----
    story.append(Paragraph('8. DETAILED CALCULATIONS', styles['H1']))
    t3 = Table(calc_table, hAlign='LEFT', colWidths=[2.2 * inch, 1.5 * inch, 2.8 * inch])
    t3.setStyle(TableStyle([
        ('SPAN', (0,0), (-1,0)),
        ('ALIGN', (0,0), (-1,0), 'CENTER'),
        ('BACKGROUND', (0,0), (-1,1), colors.HexColor('#4472C4')),
        ('TEXTCOLOR', (0,0), (-1,1), colors.whitesmoke),
        ('GRID', (0,0), (-1,-1), 0.5, colors.black),
        ('FONTNAME', (0,0), (-1,-1), MONO),
        ('FONTSIZE', (0,0), (-1,-1), 8),
        ('ROWBACKGROUNDS', (0,2), (-1,-1), [colors.white, colors.HexColor('#E7E6E6')]),
    ]))
    story.append(t3)
    story.append(PageBreak())

    # ---- 9. References ----
    story.append(Paragraph('9. REFERENCES & EQUATIONS', styles['H1']))
    refs = [
        "Hardy Cross (1932): Moment Distribution Method.",
        "ACI 318-19: Sections 22 (Strength), 24 (Serviceability), 25 (Details).",
        "Vc = 2λ√f′c · bw · d ;   Vs = Av · fy · d / s ;   ϕv = 0.75",
        "Mcr = fr · Ig / yt ;   fr = 7.5 λ √f′c",
        "Ie = (Mcr/Ma)^3 · Ig + [1 − (Mcr/Ma)^3] · Icr  (Branson).",
        "Fixed-end moments: UDL → M = wL²/12 ;  Point P@a → M_near = P a (L−a)² / L² ; M_far = P a² (L−a) / L².",
        "Shear/Moment along span: V(x)=R_L − w x − ΣP H(x−a);  M(x)=R_L x − w x²/2 − ΣP(x−a)H(x−a) − M_left."
    ]
    for r in refs:
        story.append(Paragraph(f"• {r}", styles['Normal']))
    story.append(PageBreak())

    # ---- 10. Conclusions ----
    story.append(Paragraph('10. CONCLUSIONS AND RECOMMENDATIONS', styles['H1']))
    story.append(Paragraph(info.get('Conclusions', ''), styles['Normal']))

    doc.build(story)


# ---------------------------------------------------------------------------
# Interactive main
def collect_user_inputs() -> Tuple[Dict[str, Any], BeamInput, Section, Material, List[ComboDef], Dict[str, Any]]:
    print("\n" + "=" * 70)
    print("ACI 318-19 REINFORCED CONCRETE BEAM DESIGN (REVISED MASTER)")
    print("=" * 70)
    print("\n>>> PROJECT INFORMATION")
    project = ask_string("Project name", "Structural Analysis Project")
    subject = ask_string("Subject", "RC Beam Design per ACI 318-19")
    projno  = ask_string("Project number", "2024-001")
    by      = ask_string("Designed by", "Engineer")
    checked = ask_string("Checked by", "—")
    logo    = ask_string("Logo path (optional)", LOGO_DEFAULT if os.path.exists(LOGO_DEFAULT) else "")
    plan    = ask_string("Plan View JPG path (optional)", "")
    today   = dt.date.today().isoformat()

    print("\n>>> MATERIAL PROPERTIES")
    fpc = ask_float("Concrete f'c (psi)", 4000.0)
    fy  = ask_float("Steel fy (psi)", 60000.0)

    print("\n>>> SECTION PROPERTIES")
    b = ask_float("Beam width b (inches)", 12.0)
    h = ask_float("Beam height h (inches)", 24.0)
    cover = ask_float("Clear cover to stirrups (inches)", 1.5)

    # Self-weight (kip/ft): 150 pcf * (b/12)*(h/12) * 1 ft / 1000
    self_weight_kipft = 150.0 * (b / 12.0) * (h / 12.0) / 1000.0

    print("\n>>> LOAD INPUT MODE")
    mode = ask_choice("Enter span loads as (kip/ft) or (psf+tributary width)?", ['kipft', 'psf'], 'kipft')

    # Optional global psf mode inputs
    DL_psf = LL_psf = trib_ft = 0.0
    if mode == 'psf':
        DL_psf = ask_float("Uniform DL (psf) excluding self-weight", 50.0)
        LL_psf = ask_float("Uniform LL (psf)", 40.0)
        trib_ft = ask_float("Tributary width (ft)", 10.0)

    print("\n>>> BEAM FRAMING")
    n_spans = ask_int("Number of spans", 1, min_value=1)
    joints: List[str] = []
    for j in range(n_spans + 1):
        support_name = 'left' if j == 0 else ('right' if j == n_spans else f'interior {j}')
        default_type = 'fixed' if (j == 0 or j == n_spans) else 'pinned'
        joint_type = ask_choice(f"Support at {support_name} end", ['fixed', 'pinned'], default_type)
        joints.append(joint_type)

    print("\n>>> LOADING PER SPAN")
    spans: List[Span] = []
    for i in range(n_spans):
        print(f"\n --- Span {i + 1} ---")
        L_ft = ask_float("  Span length L (feet)", 30.0)

        if mode == 'kipft':
            wD = ask_float("  Dead load wD (kip/ft)", 0.5) + self_weight_kipft
            wL = ask_float("  Live load wL (kip/ft)", 1.0)
        else:
            # Convert psf*trib(ft) to kip/ft; add self-weight to DL
            wD = (DL_psf * trib_ft) / 1000.0 + self_weight_kipft
            wL = (LL_psf * trib_ft) / 1000.0

        n_point = ask_int("  Number of point loads", 0, min_value=0)
        point_loads: List[Tuple[float, float, str]] = []
        for k in range(n_point):
            print(f"  Point load {k + 1}:")
            P  = ask_float("    Magnitude P (kips)", 10.0)
            x  = ask_float("    Location from left (feet)", L_ft / 2.0)
            lt = ask_choice("    Load type", ['d', 'l'], 'd').upper()
            point_loads.append((P, x, lt))
        spans.append(Span(L_ft, SpanLoad(wD=wD, wL=wL, point_loads=point_loads)))

    # ---- Load Combinations (default vs user-defined)
    print("\n>>> LOAD COMBINATIONS")
    use_default = ask_choice("Use default strength combos (U1:1.4D; U2:1.2D+1.6L)?", ['yes', 'no'], 'yes')
    combos: List[ComboDef] = []
    combos_meta: List[Tuple[str, str]] = []  # (name, expression pretty)

    if use_default == 'yes':
        combos = list(DEFAULT_COMBOS)
        for c in combos:
            combos_meta.append((c.name, pretty_combo(c)))
    else:
        n_combo = ask_int("Number of load combinations", 1, min_value=1)
        for i in range(n_combo):
            cname = ask_string(f"  Combo {i+1} name", f"U{i+1}")
            n_sym = ask_int("    Number of symbols in this combo (e.g., D, L)", 2, min_value=1)
            factors: Dict[str, float] = {}
            for s in range(n_sym):
                sym = ask_string(f"      Symbol {s+1} (e.g., D or L)", "D").upper()
                fac = ask_float(f"      Factor for {sym}", 1.0)
                factors[sym] = fac
            cdef = ComboDef(cname, factors)
            combos.append(cdef)
            combos_meta.append((cname, pretty_combo(cdef)))

    info = {
        'Project': project, 'Subject': subject, 'ProjectNo': projno, 'By': by,
        'Checked': checked, 'Date': today, 'LogoPath': logo, 'PlanPath': plan,
        # extra for later display
        'SelfWeight_kipft': self_weight_kipft,
        'LoadMode': mode, 'DL_psf': DL_psf, 'LL_psf': LL_psf, 'Trib_ft': trib_ft,
    }
    return info, BeamInput(spans=spans, joints=joints), Section(b=b, h=h, cover=cover), Material(fpc=fpc, fy=fy), combos, {'combos_meta': combos_meta}



def main() -> None:
    info, beam, sec, mat, combos, extras = collect_user_inputs()

    print("\n" + "=" * 70)
    print("ANALYZING...")
    print("=" * 70)

    Ec = 57000.0 * math.sqrt(mat.fpc)  # psi
    Ig = sec.b * sec.h ** 3 / 12.0     # in^4

    # --- Ultimate ENVELOPE for design (capacity checks use |.|) ---
    print("\n→ Using strength combinations:")
    for nm, expr in extras['combos_meta']:
        print(f"  • {nm}: {expr}")
    diag_env, combos_used = envelope_ultimate(beam, Ec, Ig, combos, n=1000)

    # --- Signed diagrams for plotting: pick governing ultimate combo ---
    diag_signed, gov_name = governing_signed_combo(beam, Ec, Ig, combos, n=1000, criterion="moment")

    # Locate peak |Mu| (from envelope) for design numbers
    M_abs = diag_env.M_kipin
    M_pos_max = float(np.max(M_abs))
    M_pos_loc = float(diag_env.x_in[int(np.argmax(M_abs))]) / FT2IN
    print(f"  |Mu|max: {M_pos_max:.1f} kip-in @ x ≈ {M_pos_loc:.2f} ft")
    print(f"  |V|max : {float(np.max(diag_env.V_kip)):.2f} kips")

    # --- Flexure design with multi-layer check (using depth from layout) ---
    print("\n→ Designing flexure with multi-layer check ...")
    d_guess = sec.d(1.0)
    res_pos = design_flexure(M_pos_max, sec.b, d_guess, mat.fpc, mat.fy)
    As_min_pos = As_min_flexure(sec.b, d_guess, mat.fpc, mat.fy)
    As_use_pos = max(res_pos['As'], As_min_pos)
    bot_bars = choose_bars(As_use_pos, prefer=['#5', '#6', '#7', '#8', '#9', '#10'])

    bot_layout = arrange_layers(sec.b, sec.cover, bot_bars['db'], bot_bars['n'])
    d_bot_eff  = effective_depth_from_layers(sec, bot_bars['db'], bot_layout)
    # Recompute with actual effective depth
    res_pos = design_flexure(M_pos_max, sec.b, d_bot_eff, mat.fpc, mat.fy)
    As_min_pos = As_min_flexure(sec.b, d_bot_eff, mat.fpc, mat.fy)
    As_use_pos = max(res_pos['As'], As_min_pos)

    # Mirror for negative
    res_neg = design_flexure(M_pos_max, sec.b, d_bot_eff, mat.fpc, mat.fy)
    As_min_neg = As_min_flexure(sec.b, d_bot_eff, mat.fpc, mat.fy)
    As_use_neg = max(res_neg['As'], As_min_neg)
    top_bars   = choose_bars(As_use_neg, prefer=['#5', '#6', '#7', '#8', '#9', '#10'])
    top_layout = arrange_layers(sec.b, sec.cover, top_bars['db'], top_bars['n'])

    spacing_bot = check_bar_spacing(sec.b, sec.cover, bot_bars['n'], bot_bars['db'])
    spacing_top = check_bar_spacing(sec.b, sec.cover, top_bars['n'], top_bars['db'])

    design_summary: List[str] = [
        f"Bottom steel (+M): {bot_bars['n']}-{bot_bars['size']} (As = {bot_bars['As_prov']:.3f} in²)"
        + (" — arranged in 2 layers" if len(bot_layout['layers']) > 1 else " — single layer OK"),
        f"Top steel (-M): {top_bars['n']}-{top_bars['size']} (As = {top_bars['As_prov']:.3f} in²)"
        + (" — arranged in 2 layers" if len(top_layout['layers']) > 1 else " — single layer OK"),
    ]
    if not spacing_bot['adequate']:
        design_summary.append("⚠ Bottom bar single-layer spacing inadequate → multi-layer applied.")
    if not spacing_top['adequate']:
        design_summary.append("⚠ Top bar single-layer spacing inadequate → multi-layer applied.")

    ld_bot = development_length_tension(bot_bars['db'], mat.fy, mat.fpc, sec.cover, spacing_bot['s_actual'])
    ld_top = development_length_tension(top_bars['db'], mat.fy, mat.fpc, sec.cover, spacing_top['s_actual'])

    # Flexural table
    flex_table: List[List[str]] = [[
        "Location", "Mu (k-in)", "d (in)", "As,req (in²)", "As,min (in²)", "As,use (in²)", "εt", "φ", "Governs",
    ]]
    gov_pos = "Minimum" if As_min_pos > res_pos['As'] else "Strength"
    flex_table.append([
        "Midspan (+M) [|Mu| envelope]", f"{M_pos_max:.1f}", f"{d_bot_eff:.2f}",
        f"{res_pos['As']:.3f}", f"{As_min_pos:.3f}", f"{As_use_pos:.3f}",
        f"{res_pos['eps_t']:.4f}", f"{res_pos['phi']:.3f}", gov_pos
    ])
    gov_neg = "Minimum" if As_min_neg > res_neg['As'] else "Strength"
    flex_table.append([
        "Support (-M) [|Mu| envelope]", f"{M_pos_max:.1f}", f"{d_bot_eff:.2f}",
        f"{res_neg['As']:.3f}", f"{As_min_neg:.3f}", f"{As_use_neg:.3f}",
        f"{res_neg['eps_t']:.4f}", f"{res_neg['phi']:.3f}", gov_neg
    ])

    # Shear design (zones) on envelope
    print("\n→ Designing shear reinforcement on ultimate envelope ...")
    shear_table, shear_meta = shear_schedule_three_zones(diag_env, sec, mat)

    # Service deflection (Branson) on D+L
    print("\n→ Computing service deflection (D+L) with cracked Ie (Branson) ...")
    x_serv, w_serv = compute_service_deflection_with_cracking(
        beam, Ec, Ig, sec, mat, bot_bars=bot_bars, top_bars=top_bars, n_points_per_span=1000
    )
    diag_plot = Diagrams(
        x_in=diag_signed.x_in,
        V_kip=diag_signed.V_kip,
        M_kipin=diag_signed.M_kipin,
        w_in=np.interp(diag_signed.x_in, x_serv, w_serv)
    )

    # Plots
    print("\n→ Generating plots ...")
    base_path = os.path.abspath('rc_beam_analysis')
    schematic_png = base_path + '_schematic.png'
    draw_schematic(beam, schematic_png)
    shear_png, moment_png, defl_png = plot_diagrams(diag_plot, base_path, combo_label=gov_name)
    xsec_png = base_path + '_section.png'
    stirrup_note = f"#4 2-leg @ {shear_meta['s1_prov']:.0f}\" / {shear_meta['s2_prov']:.0f}\" / {shear_meta['s3_prov']:.0f}\""
    draw_cross_section(sec, bot_bars, top_bars, xsec_png, bot_layout, top_layout, stirrup_text=stirrup_note)
    shear_design_png = plot_shear_envelope_design(diag_env, sec, mat, shear_meta, base_path)

    # Methodology text (unchanged)
    methodology_lines = [
        '<b>Elastic Analysis (Hardy Cross):</b>',
        '• Member fixed-end moments: UDL M = wL²/12; point load M_near = Pa(L−a)²/L², M_far = Pa²(L−a)/L².',
        '• Joint equilibrium with distribution and carry-over factors, iterated to tolerance.',
        '',
        '<b>Ultimate Envelopes:</b>',
        '• Strength combinations considered per Section 3.5.',
        '• |V| and |M| envelopes taken across the above combinations.',
        '',
        '<b>Service Deflection (Branson):</b>',
        "• fr = 7.5λ√f'c; Mcr = fr·Ig/yt.",
        '• Effective inertia: Ie = (Mcr/Ma)^3·Ig + [1 − (Mcr/Ma)^3]·Icr; clamped to [Icr, Ig].',
        '• w(x) from double integration of M/(Ec·Ie), zeroed at each support.',
        '',
        '<b>Flexure (ACI 318-19):</b>',
        "• a = As·fy/(0.85·f'c·b); β1 per strength reduction with f'c; c = a/β1; φ from tensile strain εt.",
        '• As,min = max(3√f′c/fy, 200/fy)·b·d.',
        '',
        '<b>Shear (ACI 318-19 §22):</b>',
        "• Vc = 2λ√f'c·bw·d; φv = 0.75. Vs = Av·fy·d/s with 2-leg #4 stirrups (Av known).",
        '• Minimum transverse reinforcement: Av/s ≥ max(0.75√f′c·bw/fy, 50·bw/fy); s ≤ min(d/2, 24 in).',
        '• Tighter support spacing applied to both end regions; midspan takes the looser spacing.',
    ]

    # 3.5 Load Combinations table data
    combos_table = [["Combo", "Expression"]]
    for nm, expr in extras['combos_meta']:
        combos_table.append([nm, expr])

    # Calc table (unchanged; we’ll add a couple of lines for load input mode)
    calc_table: List[List[str]] = [
        ['DESIGN CALCULATIONS', '', ''],
        ['Parameter', 'Value', 'Notes'],
        ('b (in)', f'{sec.b:.2f}', 'Beam width'),
        ('h (in)', f'{sec.h:.2f}', 'Overall depth'),
        ('Cover (in)', f'{sec.cover:.2f}', 'Clear cover to stirrups'),
        ('', '', ''),
        ("f'c (psi)", f'{mat.fpc:.0f}', 'Concrete strength'),
        ('fy (psi)', f'{mat.fy:.0f}', 'Steel yield strength'),
        ('Ec (ksi)', f'{Ec / 1000:.0f}', 'Concrete modulus'),
        ('Ig (in⁴)', f'{Ig:.1f}', 'Gross inertia'),
        ('', '', ''),
        ('Self weight (kip/ft)', f"{info['SelfWeight_kipft']:.4f}", '150 pcf · b · h'),
        ('Load input mode', info['LoadMode'], 'kip/ft or psf+trib width'),
    ]
    if info['LoadMode'] == 'psf':
        calc_table.extend([
            ('DL (psf) excl. self-weight', f"{info['DL_psf']:.1f}", ''),
            ('LL (psf)', f"{info['LL_psf']:.1f}", ''),
            ('Trib width (ft)', f"{info['Trib_ft']:.2f}", ''),
        ])
    calc_table.extend([
        ('', '', ''),
        ('d_eff (in)', f"{effective_depth_from_layers(sec, bot_bars['db'], bot_layout):.2f}", 'From multi-layer centroid'),
        ('Mcr (k-in)', f"{cracking_moment_rect(sec.b, sec.h, mat.fpc, mat.lam):.1f}", 'Service cracking moment'),
        ('', '', ''),
        ('Total length (ft)', f"{diag_plot.x_in[-1] / FT2IN:.2f}", 'Sum of spans'),
        ('Max |V| (kips)', f"{np.max(np.abs(diag_env.V_kip)):.2f}", 'Ultimate envelope'),
        ('Max |M| (k-in)', f"{np.max(np.abs(diag_env.M_kipin)):.1f}", 'Ultimate envelope'),
        ('Max service deflection (in)', f"{np.max(np.abs(diag_plot.w_in)):.3f}", 'Cracked Ie (Branson)'),
        ('', '', ''),
        ('Vc (kips)', f"{shear_meta['Vc']:.2f}", 'Concrete shear capacity'),
        ('φ (shear)', f"{shear_meta['phi_v']:.2f}", 'Strength reduction'),
        ('s_min_code (in)', f"{shear_meta['s_min_code']:.1f}", 'From Av/s ≥ max(...)'),
        ('s_max per code (in)', f"{shear_meta['s_max']:.1f}", '≤ min(d/2, 24 in)'),
        ('', '', ''),
        ('Bottom bars', f"{bot_bars['n']}-{bot_bars['size']}", f"As = {bot_bars['As_prov']:.3f} in²"),
        ('Top bars', f"{top_bars['n']}-{top_bars['size']}", f"As = {top_bars['As_prov']:.3f} in²"),
        ('ld (bottom)', f"{ld_bot:.1f}", 'Development length (approx)'),
        ('ld (top)', f"{ld_top:.1f}", 'Development length (approx)'),
        ('', '', ''),
        ('Stirrups Zone 1', f"#4 @ {shear_meta['s1_prov']:.0f}\"", f"x = 0 to {shear_meta['z_len']:.1f} ft"),
        ('Stirrups Zone 2', f"#4 @ {shear_meta['s2_prov']:.0f}\"", f"x = {shear_meta['z_len']:.1f} to {2 * shear_meta['z_len']:.1f} ft"),
        ('Stirrups Zone 3', f"#4 @ {shear_meta['s3_prov']:.0f}\"", f"x = {2 * shear_meta['z_len']:.1f} to {3 * shear_meta['z_len']:.1f} ft"),
    ])

    info['Scope'] = (
        "This package presents analysis and design of a reinforced concrete beam per ACI 318-19. "
        "Ultimate shear/moment envelopes are formed from the load combinations listed in Section 3.5. "
        "Service deflection uses Branson effective inertia."
    )
    info['Conclusions'] = (
        "The proposed reinforcement satisfies ACI 318-19 strength checks for flexure and shear based on the analyzed loads. "
        "Verify serviceability limits, development/splice lengths, and constructibility/spacing per ACI 318-19 Ch. 25 in final detailing."
    )

    print("\n→ Building PDF report ...")
    out_pdf_path = os.path.abspath('RC_Beam_Design_Report_ACI318.pdf')
    build_pdf_report(
        out_pdf_path, info, diag_plot, sec, mat, flex_table, shear_table, design_summary,
        schematic_png, shear_png, moment_png, defl_png, xsec_png, calc_table,
        methodology_lines, stirrup_note, shear_design_png,
        combo_label=gov_name, plan_view_jpg=info.get('PlanPath'),
        combos_table=combos_table
    )

    print("\n" + "=" * 70)
    print("ANALYSIS COMPLETE!")
    print("=" * 70)
    print(f"\n✓ PDF Report: {out_pdf_path}")
    print("\n✓ Load combinations used:")
    for nm, expr in extras['combos_meta']:
        print(f"  • {nm}: {expr}")
    print("\n✓ Design Summary:")
    for line in design_summary:
        print(f"  • {line}")
    print("\n✓ Stirrup Schedule:")
    for row in shear_table[1:]:
        zone_name = row[0]; spacing = row[6]
        print(f"  • {zone_name}: #4 stirrups @ {spacing} o.c.")
    print(f"\n✓ Max service deflection (cracked Ie): {np.max(np.abs(diag_plot.w_in)):.3f} in")
    print("  (Deflection based on service moments and Branson effective inertia.)")
    print("\n" + "=" * 70 + "\n")



if __name__ == '__main__':
    main()



ACI 318-19 REINFORCED CONCRETE BEAM DESIGN (REVISED MASTER)

>>> PROJECT INFORMATION
Project name [Structural Analysis Project]: 
Subject [RC Beam Design per ACI 318-19]: 
Project number [2024-001]: 
Designed by [Engineer]: 
Checked by [—]: 
Logo path (optional) [/content/stvlogo23.png]: 
Plan View JPG path (optional) []: 

>>> MATERIAL PROPERTIES
Concrete f'c (psi) [4000.0]: 
Steel fy (psi) [60000.0]: 

>>> SECTION PROPERTIES
Beam width b (inches) [12.0]: 
Beam height h (inches) [24.0]: 
Clear cover to stirrups (inches) [1.5]: 

>>> LOAD INPUT MODE
Enter span loads as (kip/ft) or (psf+tributary width)? (kipft/psf) [kipft]: 

>>> BEAM FRAMING
Number of spans [1]: 
Support at left end (fixed/pinned) [fixed]: 
Support at right end (fixed/pinned) [fixed]: 

>>> LOADING PER SPAN

 --- Span 1 ---
  Span length L (feet) [30.0]: 
  Dead load wD (kip/ft) [0.5]: 
  Live load wL (kip/ft) [1.0]: 
  Number of point loads [0]: 

>>> LOAD COMBINATIONS
Use default strength combos (U1:1.4D; U2:1.2D+1